## Simulation 
We begin by specifying packages, reading in the dataset and generating some functions that we will use repeatedly to calculate poverty rates and gini coefficients as well as to generate simple percentage changes.

In the dataframe, we have spmu unit years as the level of the dataset. All values are averaged across 2018-2020.

In [1]:
import microdf as mdf
import pandas as pd
import numpy as np
import us
import plotly.express as px
import plotly.graph_objects as go
from ubicenter import format_fig

person_sim = pd.read_csv(
    "C:\\Users\\John Walker\\Desktop\\person_sim.csv.gz",
    # "jb/data/person_sim.csv.gz",
    compression="gzip")
person_sim.rename(columns={'scenario':'Scenario'}, inplace=True)

In [2]:
# Define a function to calculate poverty rates from the poverty flag
def pov(data, group):
    return pd.DataFrame(
        mdf.weighted_mean(data, "poverty_flag", "asecwt", groupby=group)
    )

def deep_pov(data, group):
    return pd.DataFrame(
        mdf.weighted_mean(data, "deep_poverty_flag", "asecwt", groupby=group)
)

# Define function to generate gini coefficients
def gin(data, group):
    return pd.DataFrame(
        data.groupby(group).apply(
            lambda x: mdf.gini(x, "spmtotres", "asecwt")
        )
    )

# Define percentage change function
def percent_change(new, old):
    return 100 * (new - old) / old

We generate poverty rates for the total population and by demographics of interest, namely sex, race, whether one is a child (under 6) and by state. We similarly generate gini coefficients for the total population and by state.

In [59]:
# Poverty rates by demographics of interest
poverty_rate = pov(person_sim, ["scenario", "ca"]) # Overall poverty rate
poverty_rate_sex = pov(person_sim, ["scenario", "ca", "sex"]) # Poverty rates by sex
poverty_rate_race_hispan = pov(person_sim, ["scenario", "ca", "race_hispan"])  # Poverty rates by race
poverty_rate_age = pov(person_sim, ["scenario", "ca", "age_cat"]) # Poverty rate by child age
poverty_rate_child = pov(person_sim, ["scenario", "ca", "child_6"]) # Poverty rate by child age

# State-based poverty rates
poverty_rate_state = pov(person_sim, ["scenario", "ca", "state"])

# Rename constructed poverty_rates
poverty_rates = [
    poverty_rate,
    poverty_rate_sex,
    poverty_rate_race_hispan,
    poverty_rate_state,
    poverty_rate_age,
]
for i in poverty_rates:
    i.rename({0: "poverty_rate"}, axis=1, inplace=True)

for j in poverty_rates:
    j = j.reset_index(inplace=True)

In [60]:
# Deep poverty rates by demographics of interest
deep_poverty_rate = deep_pov(person_sim, ["scenario", "ca"]) # Overall poverty rate
deep_poverty_rate_sex = deep_pov(person_sim, ["scenario", "ca", "sex"]) # Poverty rates by sex
deep_poverty_rate_race_hispan = deep_pov(person_sim, ["scenario", "ca", "race_hispan"])  # Poverty rates by race
deep_poverty_rate_age = deep_pov(person_sim, ["scenario", "ca", "age_cat"]) # Poverty rate by child age
deep_poverty_rate_child = pov(person_sim, ["scenario", "ca", "child_6"]) # Poverty rate by child age

# State-based deep poverty rates
deep_poverty_rate_state = deep_pov(person_sim, ["scenario", "ca", "state"])

# Rename constructed poverty_rates
deep_poverty_rates = [
    deep_poverty_rate,
    deep_poverty_rate_sex,
    deep_poverty_rate_race_hispan,
    deep_poverty_rate_state,
    deep_poverty_rate_age,
]
for i in deep_poverty_rates:
    i.rename({0: "deep_poverty_rate"}, axis=1, inplace=True)

for j in deep_poverty_rates:
    j = j.reset_index(inplace=True)

In [30]:
# Gini coefficients and state-based heterogenous gini coefficients
gini = gin(person_sim, ["scenario", "ca"])
gini_state = gin(person_sim, ["scenario", "ca", "state"])

# Rename constructed gini coefficients
ginis = [
    gini,
    gini_state,
]
for i in ginis:
    i.rename({0: "gini_coefficient"}, axis=1, inplace=True)
for j in ginis:
    j = j.reset_index(inplace=True)

In [69]:
poverty_rate.columns

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [14]:
# Poverty plot
# Use pivot table to construct difference / percentage change in pov measures 
pov_wide = poverty_rate[poverty_rate.scenario!="baseline"].pivot_table("poverty_rate","scenario","ca")
pov_wide["baseline"] = poverty_rate[poverty_rate.scenario == "baseline"].poverty_rate.values[0]
pov_wide = pov_wide.reset_index()
pov_wide.columns = ["scenario", "child care", "child allowance", "baseline"]
pov_wide["pdif_base_cc"] = percent_change(pov_wide["child care"],pov_wide["baseline"])
pov_wide["pdif_base_ca"] = percent_change(pov_wide["child allowance"],pov_wide["baseline"])
pov_wide["pdif_ca_cc"] = percent_change(pov_wide["child allowance"],pov_wide["child care"])

# Prepare dataframe for plot output
no_ca_rows = {"scenario":pov_wide["scenario"], "Transfer type":"Child care", "Percentage difference in poverty from baseline":pov_wide["pdif_base_cc"]}
no_ca_rows = pd.DataFrame(no_ca_rows)
row_1 = no_ca_rows[no_ca_rows.scenario == "cc_replacement"]
row_2 = no_ca_rows[no_ca_rows.scenario == "low_cc_full"]
row_3 = no_ca_rows[no_ca_rows.scenario == "high_cc_full"]

ca_rows = {"scenario":pov_wide["scenario"], "Transfer type":"Child allowance", "Percentage difference in poverty from baseline":pov_wide["pdif_base_ca"]}
ca_rows = pd.DataFrame(ca_rows)
row_4 = ca_rows[ca_rows.scenario == "cc_replacement"]
row_5 = ca_rows[ca_rows.scenario == "low_cc_full"]
row_6 = ca_rows[ca_rows.scenario == "high_cc_full"]

# Append the rows
row_2 = row_1.append(row_2)
row_3 = row_2.append(row_3)
row_4 = row_3.append(row_4)
row_5 = row_4.append(row_5)
row_6 = row_5.append(row_6)
pov_long = row_6

# Rename scenarios
pov_long.loc[pov_long.scenario == "cc_replacement", "scenario"] = "Expense replacement"
pov_long.loc[pov_long.scenario == "high_cc_full", "scenario"] = "High quality"
pov_long.loc[pov_long.scenario == "low_cc_full", "scenario"] = "Base quality" 

pov_plot = px.bar(pov_long, x="scenario", y="Percentage difference in poverty from baseline", color="Transfer type", barmode="group",color_discrete_map={"Child care":"steelblue","Child allowance":"grey"})
format_fig(pov_plot)


In [15]:
### Deep poverty
# Use pivot table to construct difference / percentage change in pov measures 
deep_pov_wide = deep_poverty_rate[deep_poverty_rate.scenario!="baseline"].pivot_table("deep_poverty_rate","scenario","ca")
deep_pov_wide["baseline"] = deep_poverty_rate[deep_poverty_rate.scenario == "baseline"].deep_poverty_rate.values[0]
deep_pov_wide = deep_pov_wide.reset_index()
deep_pov_wide.columns = ["scenario", "child care", "child allowance", "baseline"]
deep_pov_wide["pdif_base_cc"] = percent_change(deep_pov_wide["child care"],deep_pov_wide["baseline"])
deep_pov_wide["pdif_base_ca"] = percent_change(deep_pov_wide["child allowance"],deep_pov_wide["baseline"])
deep_pov_wide["pdif_ca_cc"] = percent_change(deep_pov_wide["child allowance"],deep_pov_wide["child care"])

# Prepare dataframe for plot output
no_ca_rows = {"scenario":deep_pov_wide["scenario"], "Transfer type":"Child care", "Percentage difference in poverty from baseline":deep_pov_wide["pdif_base_cc"]}
no_ca_rows = pd.DataFrame(no_ca_rows)
row_1 = no_ca_rows[no_ca_rows.scenario == "cc_replacement"]
row_2 = no_ca_rows[no_ca_rows.scenario == "low_cc_full"]
row_3 = no_ca_rows[no_ca_rows.scenario == "high_cc_full"]

ca_rows = {"scenario":deep_pov_wide["scenario"], "Transfer type":"Child allowance", "Percentage difference in poverty from baseline":deep_pov_wide["pdif_base_ca"]}
ca_rows = pd.DataFrame(ca_rows)
row_4 = ca_rows[ca_rows.scenario == "cc_replacement"]
row_5 = ca_rows[ca_rows.scenario == "low_cc_full"]
row_6 = ca_rows[ca_rows.scenario == "high_cc_full"]

# Append the rows
row_2 = row_1.append(row_2)
row_3 = row_2.append(row_3)
row_4 = row_3.append(row_4)
row_5 = row_4.append(row_5)
row_6 = row_5.append(row_6)
pov_long = row_6

# Rename scenarios
pov_long.loc[pov_long.scenario == "cc_replacement", "scenario"] = "Expense replacement"
pov_long.loc[pov_long.scenario == "high_cc_full", "scenario"] = "High quality"
pov_long.loc[pov_long.scenario == "low_cc_full", "scenario"] = "Base quality" 

pov_plot = px.bar(pov_long, x="scenario", y="Percentage difference in poverty from baseline", color="Transfer type", barmode="group",color_discrete_map={"Child care":"steelblue","Child allowance":"grey"})
format_fig(pov_plot)


In [62]:
poverty_rate_child

0
scenario       ca    child_6          
baseline       False False    0.126100
                     True     0.149787
cc_replacement False False    0.124124
                     True     0.139226
               True  False    0.120445
                     True     0.115222
high_cc_full   False False    0.106706
                     True     0.013208
               True  False    0.106664
                     True     0.012909
low_cc_full    False False    0.111094
                     True     0.039738
               True  False    0.110938
                     True     0.038074

In [61]:
# Poverty plot age
# Use pivot table to construct difference / percentage change in pov measures 
child_pov_wide = poverty_rate_child[poverty_rate_child.scenario!="baseline"].pivot_table("poverty_rate","scenario","ca", "child_6")
child_pov_wide["baseline"] = poverty_rate_child[poverty_rate_child.scenario == "baseline"].poverty_rate.values[0]
child_pov_wide = child_pov_wide.reset_index()
child_pov_wide.columns = ["scenario", "child care", "child allowance", "baseline"]
child_pov_wide["pdif_base_cc"] = percent_change(child_pov_wide["child care"],child_pov_wide["baseline"])
child_pov_wide["pdif_base_ca"] = percent_change(child_pov_wide["child allowance"],child_pov_wide["baseline"])
child_pov_wide["pdif_ca_cc"] = percent_change(child_pov_wide["child allowance"],child_pov_wide["child care"])

# Prepare dataframe for plot output
no_ca_rows = {"scenario":child_pov_wide["scenario"], "Transfer type":"Child care", "Percentage difference in poverty from baseline":child_pov_wide["pdif_base_cc"]}
no_ca_rows = pd.DataFrame(no_ca_rows)
row_1 = no_ca_rows[no_ca_rows.scenario == "cc_replacement"]
row_2 = no_ca_rows[no_ca_rows.scenario == "low_cc_full"]
row_3 = no_ca_rows[no_ca_rows.scenario == "high_cc_full"]

ca_rows = {"scenario":child_pov_wide["scenario"], "Transfer type":"Child allowance", "Percentage difference in poverty from baseline":child_pov_wide["pdif_base_ca"]}
ca_rows = pd.DataFrame(ca_rows)
row_4 = ca_rows[ca_rows.scenario == "cc_replacement"]
row_5 = ca_rows[ca_rows.scenario == "low_cc_full"]
row_6 = ca_rows[ca_rows.scenario == "high_cc_full"]

# Append the rows
row_2 = row_1.append(row_2)
row_3 = row_2.append(row_3)
row_4 = row_3.append(row_4)
row_5 = row_4.append(row_5)
row_6 = row_5.append(row_6)
pov_long = row_6

# Rename scenarios
pov_long.loc[pov_long.scenario == "cc_replacement", "scenario"] = "Expense replacement"
pov_long.loc[pov_long.scenario == "high_cc_full", "scenario"] = "High quality"
pov_long.loc[pov_long.scenario == "low_cc_full", "scenario"] = "Base quality" 

pov_plot = px.bar(pov_long, x="scenario", y="Percentage difference in poverty from baseline", color="Transfer type", barmode="group",color_discrete_map={"Child care":"steelblue","Child allowance":"grey"})
format_fig(pov_plot)


AttributeError: 'DataFrame' object has no attribute 'scenario'

In [45]:
### Gini
# Use pivot table to construct difference / percentage change in pov measures 
gini_wide = gini[gini.scenario!="baseline"].pivot_table("gini_coefficient","scenario","ca")
gini_wide["baseline"] = gini[gini.scenario == "baseline"].gini_coefficient.values[0]
gini_wide = gini_wide.reset_index()
gini_wide.columns = ["scenario", "child care", "child allowance", "baseline"]
gini_wide["pdif_base_cc"] = percent_change(gini_wide["child care"],gini_wide["baseline"])
gini_wide["pdif_base_ca"] = percent_change(gini_wide["child allowance"],gini_wide["baseline"])
gini_wide["pdif_ca_cc"] = percent_change(gini_wide["child allowance"],gini_wide["child care"])

# Prepare dataframe for plot output
no_ca_rows = {"scenario":gini_wide["scenario"], "Transfer type":"Child care", "Percentage difference in Gini coefficient from baseline":gini_wide["pdif_base_cc"]}
no_ca_rows = pd.DataFrame(no_ca_rows)
row_1 = no_ca_rows[no_ca_rows.scenario == "cc_replacement"]
row_2 = no_ca_rows[no_ca_rows.scenario == "low_cc_full"]
row_3 = no_ca_rows[no_ca_rows.scenario == "high_cc_full"]

ca_rows = {"scenario":gini_wide["scenario"], "Transfer type":"Child allowance", "Percentage difference in Gini coefficient from baseline":gini_wide["pdif_base_ca"]}
ca_rows = pd.DataFrame(ca_rows)
row_4 = ca_rows[ca_rows.scenario == "cc_replacement"]
row_5 = ca_rows[ca_rows.scenario == "low_cc_full"]
row_6 = ca_rows[ca_rows.scenario == "high_cc_full"]

# Append the rows
row_2 = row_1.append(row_2)
row_3 = row_2.append(row_3)
row_4 = row_3.append(row_4)
row_5 = row_4.append(row_5)
row_6 = row_5.append(row_6)
pov_long = row_6

# Rename scenarios
pov_long.loc[pov_long.scenario == "cc_replacement", "scenario"] = "Expense replacement"
pov_long.loc[pov_long.scenario == "high_cc_full", "scenario"] = "High quality"
pov_long.loc[pov_long.scenario == "low_cc_full", "scenario"] = "Base quality" 

pov_plot = px.bar(pov_long, x="scenario", y="Percentage difference in Gini coefficient from baseline", color="Transfer type", barmode="group",color_discrete_map={"Child care":"steelblue","Child allowance":"grey"})
format_fig(pov_plot)


We then output state-based poverty rate and gini percentage changes to reflect the impact of the simulation. 

scenario              baseline cc_replacement           high_cc_full  \
ca                       False          False     True         False   
state                                                                  
Alabama               0.127468       0.126259  0.120791     0.102628   
Alaska                0.132965       0.131078  0.125391     0.094711   
Arizona               0.119701       0.116080  0.106740     0.090913   
Arkansas              0.130987       0.128325  0.120045     0.103000   
California            0.174174       0.172219  0.166576     0.136286   
Colorado              0.116944       0.114349  0.113578     0.093067   
Connecticut           0.113332       0.111758  0.110628     0.087151   
Delaware              0.105911       0.102770  0.097914     0.086105   
District of Columbia  0.169771       0.167536  0.160920     0.142059   
Florida               0.159601       0.156689  0.150109     0.129471   
Georgia               0.136660       0.134254  0.129246     0.103916   
Hawaii                0.137703       0.135924  0.128062     0.106934   
Idaho                 0.083094       0.081964  0.077663     0.063712   
Illinois              0.113958       0.111418  0.105406     0.088246   
Indiana               0.103981       0.102824  0.096494     0.075486   
Iowa                  0.074519       0.072602  0.069927     0.054115   
Kansas                0.075302       0.072967  0.070127     0.063359   
Kentucky              0.112288       0.111472  0.106124     0.095469   
Louisiana             0.165583       0.162260  0.154244     0.126427   
Maine                 0.093941       0.091307  0.089332     0.076305   
Maryland              0.124856       0.120221  0.121439     0.104525   
Massachusetts         0.114918       0.112259  0.108753     0.097114   
Michigan              0.100550       0.099210  0.094052     0.079346   
Minnesota             0.065550       0.064345  0.061408     0.054521   
Mississippi           0.154012       0.152917  0.143865     0.123675   
Missouri              0.088932       0.087982  0.081324     0.071937   
Montana               0.090210       0.088316  0.085664     0.073139   
Nebraska              0.091352       0.086408  0.083103     0.070669   
Nevada                0.135462       0.133438  0.123107     0.105098   
New Hampshire         0.084666       0.079382  0.079345     0.064278   
New Jersey            0.129822       0.126650  0.124276     0.105103   
New Mexico            0.136001       0.135767  0.128677     0.105372   
New York              0.151846       0.147671  0.143332     0.116787   
North Carolina        0.135841       0.133010  0.123163     0.102829   
North Dakota          0.095881       0.092904  0.088491     0.073530   
Ohio                  0.103376       0.101311  0.095807     0.078988   
Oklahoma              0.106826       0.105936  0.101229     0.085099   
Oregon                0.112526       0.110430  0.105006     0.093767   
Pennsylvania          0.111484       0.109140  0.105990     0.085566   
Rhode Island          0.081049       0.080227  0.078376     0.061323   
South Carolina        0.133161       0.129451  0.123233     0.109477   
South Dakota          0.090533       0.089885  0.085615     0.070910   
Tennessee             0.111710       0.109491  0.103327     0.086972   
Texas                 0.140665       0.136442  0.131118     0.104464   
Utah                  0.082092       0.080179  0.076899     0.064585   
Vermont               0.105788       0.102500  0.100875     0.092524   
Virginia              0.117839       0.112749  0.109017     0.089638   
Washington            0.083740       0.082462  0.078328     0.064484   
West Virginia         0.125399       0.123743  0.116789     0.101613   
Wisconsin             0.078747       0.077000  0.072476     0.060932   
Wyoming               0.100047       0.099021  0.092544     0.075421   

scenario                       low_cc_full            
ca                       True        False     True   
s

In [18]:
# State analysis
# Create pivot table to interpret state-based poverty effects
state_pov = poverty_rate_state.pivot_table(
    values="poverty_rate", index="state", columns=["scenario","ca"]
)
state_pov = state_pov.reset_index()
deep_state_pov = deep_poverty_rate_state.pivot_table(
    values="poverty_rate", index="state", columns=["scenario","ca"]
)
# Create pivot table to interpret state-based gini effects
state_gini = gini_state.pivot_table(
    values="gini_coefficient", index="state", columns=["scenario", "ca"]
)

us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'American Samoa': 'AS',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Guam': 'GU',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'
}
state_pov["state_code"]= state_pov.state.map(us_state_abbrev)
px.choropleth(locations=state_pov.state_code,locationmode="USA-states",color=state_pov.baseline,scope="usa")

KeyError: 'poverty_rate'

In [ ]:
state_pov

In [26]:

"""
# Generate state-based poverty rate percentage changes
state_pov["poverty_change_cc"] =  state_pov.cc_replacement - state_pov.baseline
state_pov["poverty_change_flat"] = state_pov.child_allowance - state_pov.baseline
state_pov["poverty_change_pc_cc"] = state_pov.poverty_change_cc - state_pov.baseline
state_pov["poverty_change_pc_flat"] = (
    state_pov.poverty_change_flat - state_pov.baseline
)

# Construct state-based gini coefficient percentage changes
state_gini["gini_change_cc"] = state_gini.cc_replacement - state_gini.baseline
state_gini["gini_change_flat"] = state_gini.child_allowance - state_gini.baseline
state_gini["gini_change_pc_cc"] = percent_change(
    state_gini.gini_change_cc, state_gini.baseline
)
state_gini["gini_change_pc_flat"] = percent_change(
    state_gini.gini_change_flat, state_gini.baseline
)

# Re-arrange and present pivot tables, descending by % change
# in poverty rate
state_pov.sort_values(by="poverty_change_pc_flat", ascending=True)
state_gini.sort_values(by="gini_change_pc_flat", ascending=True)
"""

'\n# Generate state-based poverty rate percentage changes\nstate_pov["poverty_change_cc"] =  state_pov.cc_replacement - state_pov.baseline\nstate_pov["poverty_change_flat"] = state_pov.child_allowance - state_pov.baseline\nstate_pov["poverty_change_pc_cc"] = state_pov.poverty_change_cc - state_pov.baseline\nstate_pov["poverty_change_pc_flat"] = (\n    state_pov.poverty_change_flat - state_pov.baseline\n)\n\n# Construct state-based gini coefficient percentage changes\nstate_gini["gini_change_cc"] = state_gini.cc_replacement - state_gini.baseline\nstate_gini["gini_change_flat"] = state_gini.child_allowance - state_gini.baseline\nstate_gini["gini_change_pc_cc"] = percent_change(\n    state_gini.gini_change_cc, state_gini.baseline\n)\nstate_gini["gini_change_pc_flat"] = percent_change(\n    state_gini.gini_change_flat, state_gini.baseline\n)\n\n# Re-arrange and present pivot tables, descending by % change\n# in poverty rate\nstate_pov.sort_values(by="poverty_change_pc_flat", ascending=True

## Visualizations


In [ ]:
# Poverty rate by sim
pov_plot = px.bar(poverty_rate,x="sim",y="poverty_rate")
format_fig(pov_plot)

In [ ]:
# Deep poverty by sim
deep_pov_plot = px.bar(deep_poverty_rate,x="sim",y="poverty_rate")
format_fig(deep_pov_plot)

In [13]:
# Child poverty by sim
child_pov_plot = px.bar(poverty_rate_child,x="sim",y="poverty_rate")
format_fig(child_pov_plot)

In [29]:
# Deep child poverty by sim 
deep_pov_child_plot = px.bar(deep_poverty_rate_child,x="sim",y="poverty_rate")
format_fig(deep_pov_child_plot)

,poverty_rate
sim,
baseline,0.127830
high_ca,0.000000
high_xpns,0.000000
low_ca,0.000000
low_xpns,0.000000
replace_ca,0.120064
replace_xpns,0.125228
